In [ ]:
!pip install metric_learn
from datapreparation.profile_parser import *
from metric_learn_algos.metric_learn_algos import *
import pandas as pd
import numpy as np
from  sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [123]:
x={'2':'ootwo','21':'one'}
sorted(x.items(),key=lambda kv:kv[1])

[('21', 'one'), ('2', 'ootwo')]

In [113]:
def similarity_scorer(x,original_df,profile_id,threshold,pca=False,includeAmount=False): 
#     x=x.copy()
    profile_index=np.where(original_df['profile_id']==profile_id)
    profile_data=x.iloc[profile_index]
    
    if not includeAmount:
        x=x.drop('Amount1',axis=1)
        profile_data=profile_data.drop('Amount1',axis=1)
        
    if pca:
        pca = PCA(n_components=30)
        df2=pca.fit_transform(x)

        df2=pd.DataFrame(df2)
        sim=cosine_similarity(df2.iloc[profile_index],df2)
    else:
        sim=cosine_similarity(profile_data,x)

    indices=np.where(sim[0]>=threshold)

    similarity_scores={}
    for i in indices[0]:
        similarity_scores[original_df.iloc[i]['profile_id']]=sim[0][i]
    
    print(sorted(similarity_scores.items(), key=lambda kv: kv[1],reverse=True))
    return sorted(similarity_scores.items(), key=lambda kv: kv[1],reverse=True)

def similarity_scorer_with_metric_learn(X,Y,raw_df,profile_id,threshold,algo):
    if algo =='LMNN':
        X_trans=pd.DataFrame(metric_lmnn(X,Y))
    elif algo =='NCA':
        X_trans=pd.DataFrame(metric_nca(X,Y))
    elif algo =='LFDA':
        X_trans=pd.DataFrame(metric_lfda(X,Y))
    elif algo =='MLKR':
        X_trans=pd.DataFrame(metric_mlkr(X,Y))
    profile_index=np.where(raw_df['profile_id']==profile_id)
    profile_data=X_trans.iloc[profile_index]
    sim=cosine_similarity(profile_data,X_trans)
    indices=np.where(sim[0]>=threshold)
    similarity_scores={}
    for i in indices[0]:
        similarity_scores[raw_df.iloc[i]['profile_id']]=sim[0][i]
    return sorted(similarity_scores.items(), key=lambda kv: kv[1],reverse=True)

In [94]:
def read_from_mc(managed_content_path, token):
    from io import StringIO
    import requests
    headers={'Authorization': 'Bearer {}'.format(token)}
    cleansed_data=requests.get(managed_content_path,headers=headers)
    cleansed_data_df=pd.read_csv(StringIO(cleansed_data.text))
    return cleansed_data_df
    
token='eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJjb2duaXRpdmVzY2FsZS5jb20iLCJhdWQiOiJjb3J0ZXgiLCJzdWIiOiJzc3dldGFuayIsInRlbmFudCI6ImFjY2VsZXJhdG9ycy1kZXYiLCJiZWFyZXIiOiJwdWJsaWMiLCJrZXkiOiJMazU4NkpRdGc3OE01OG5RakcwVkIzMTFOb1g0Mnc1SyIsImV4cCI6MTU2MzQzODYzNCwiYWNsIjp7Ii4qIjpbIlJFQUQiLCJSVU4iLCJXUklURSIsIkRFTEVURSJdLCIvdjMvYWdlbnRzL2Vudmlyb25tZW50cy8uKiI6WyJERU5ZIl0sIi92Mi9hZG1pbi8uKiI6WyJERU5ZIl0sIi92Mi90ZW5hbnRzLy4qIjpbIkRFTlkiXSwiL3YyL3RlbmFudHMvY3VycmVudC11c2VyLWRldGFpbHMiOlsiUkVBRCJdLCIvdjIvYWNjb3VudHMvLioiOlsiREVOWSJdLCIvdjMvY2F0YWxvZy8uKiI6WyJSRUFEIiwiUlVOIiwiV1JJVEUiLCJERUxFVEUiXSwiL3YyL2FjY291bnRzL3Rva2Vucy8uKiI6WyJSRUFEIiwiUlVOIiwiV1JJVEUiXSwiL3YyL3RlbmFudHMvc2VjcmV0cy8uKiI6WyJSRUFEIiwiUlVOIiwiV1JJVEUiXSwiL3YzL2FnZW50cy9lbnZpcm9ubWVudHMvY29ydGV4L2RlZmF1bHQiOlsiUkVBRCIsIlJVTiIsIldSSVRFIl19LCJpYXQiOjE1NjIyMjkwMzR9.uAPxZSI7kf4K1IUENF5k4ajH4IO3mGDc8MqHHQGTRH0'
raw_data_path='https://api.cortex.insights.ai/v2/content/accelerators/profile-similarity/rawdata'
cleansed_data_path='https://api.cortex.insights.ai/v2/content/accelerators/profile-similarity/cleanseddata'
raw_df=read_from_mc(raw_data_path,token)
cleansed_df=read_from_mc(cleansed_data_path,token)


In [95]:
raw_df.head()

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,[],['Eduardo Brayner'],General Merchandisers,0015I11484m7u87wBuD
1,10580000,Astrid Schmaus,Amplify - Enterprise(Survellance Insights),Use Case Confirmed,Exxon Mobil,[],['Ridwan Zurawik'],Petroleum Refining,0017K23751y0u72fZpZ
2,21483000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Berkshire Hathaway,[],['Courtney Bodey'],Insurance: Property and Casualty (Stock),0017O90702b2u22mRuO
3,10089000,Paris Poma,Engage - Enterprise(Customer360),Prospecting,Apple,[],['Axel Whitsel'],"Computers, Office Equipment",0014B28932p5u37rWgX
4,9141000,Lisa Kalafatis,Cortex Enterprise(MachineLearning),Negotiations,UnitedHealth Group,[],['John Santelli'],Health Care: Insurance and Managed Care,0015R72262j6u33kWxP


In [96]:
raw_df.sort_values(by='industry_name')

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
357,50000,Gilda Cubine,Cortex QuickStart(AI Develpoment),Negotiations,New York,[],['Esther Fannell'],"Advertising, marketing",0010C38260t0u55xUrI
186,50000,Elba Saylors,Cortex QuickStart(AI Develpoment),Negotiations,New York,[],['Jason Cohen'],"Advertising, marketing",0010V37393k9u33zQiU
50,50000,Astrid Schmaus,Cortex QuickStart(AI Develpoment),Negotiations,United Technologies,[],['Brian Galovich'],Aerospace and Defense,0015Y93284g1u93qFrH
223,21755000,Kyle Pouk,Cortex Enterprise(MachineLearning),Negotiations,New York,[],['Mark Husnick'],Aerospace and Defense,0010K83574y4u78xIvL
26,50000,Erick Pezley,Cortex QuickStart(MachineLearning),Needs Identification,Boeing,[],['Ted Colbert III'],Aerospace and Defense,0011O56305d9u14tJmL
379,50000,Stacie Khensovan,Engage - QuickStart(Coversational AI),Prospecting,Newport News,[],['Bharat Amin'],Aerospace and Defense,0014L50256f7u62bQcG
805,19270000,Johnnie Surrell,Cortex Enterprise(MachineLearning),Negotiations,Thousand Oaks,[],['Eve Whitecotton'],Aerospace and Defense,0015G59474o5u66rDyP
883,50000,Lucius Mcgahen,Engage - QuickStart(Customer360),Proposed Solution,Charlotte,[],['Jakob Hain'],Aerospace and Defense,0011Q01837x9u77kOzN
974,50000,Yunus Caley,Cortex QuickStart(MachineLearning),Needs Identification,Chantilly,[],['Khadeeja Issler'],Aerospace and Defense,0019S08815j9u54nMbI
954,50000,Kasey Condict,Amplify - QuickStart(Process Insights),Confirming Value with Power,McLean,[],['Lacy Dontas'],Aerospace and Defense,0016L57286t0u17gDaP


In [97]:
raw_df[raw_df.industry_name=='Computers, Office Equipment']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
3,10089000,Paris Poma,Engage - Enterprise(Customer360),Prospecting,Apple,[],['Axel Whitsel'],"Computers, Office Equipment",0014B28932p5u37rWgX
34,50000,Janice Cerniglia,Cortex QuickStart(MachineLearning),Needs Identification,Dell Technologies,[],['Axel Whitsel'],"Computers, Office Equipment",0018I84600a7u13kRsG
57,24521000,Lilly Raymo,Cortex Enterprise(MachineLearning),Negotiations,HP,[],['Chloe Arrington'],"Computers, Office Equipment",0019O42201r2u71vYsJ
105,50000,Kameron Hanzl,Cortex QuickStart(AI Develpoment),Negotiations,Palo Alto,[],['Archana Deskus'],"Computers, Office Equipment",0013X16242j5u66jMbS
156,50000,Valentine Unnold,Engage - QuickStart(Coversational AI),Prospecting,San Jose,[],['Neo Comings'],"Computers, Office Equipment",0017U30636n8u39sJcX
289,50000,Ernie Santoscoy,Engage - QuickStart(Customer360),Proposed Solution,Norwalk,[],['Zane Lovings'],"Computers, Office Equipment",0010Z59426c3u52uAnH
430,50000,Elias Maresh,Cortex QuickStart(MachineLearning),Needs Identification,Atlanta,[],['William VanCuren'],"Computers, Office Equipment",0019G81491h1u71uYuU
493,50000,John Sebastiano,Engage - QuickStart(Coversational AI),Prospecting,Sunnyvale,[],['Bob Houghton'],"Computers, Office Equipment",0019H41435m6u21aRnP
549,50000,Loraine Killeen,Cortex QuickStart(MachineLearning),Needs Identification,North Canton,[],['Tatiana Vandyck'],"Computers, Office Equipment",0012U67123t0u60mGlE
655,50000,Yesenia Mention,Cortex QuickStart(AI Develpoment),Negotiations,Stamford,[],['Louise Lacaze'],"Computers, Office Equipment",0016M91499n1u07eAoL


In [144]:
cleansed_df.columns

Index(['Amount1', 'account_name', 'contacts', 'profile_id',
       'competitor_names_AIG', 'competitor_names_AT&T',
       'competitor_names_Aetna', 'competitor_names_Akron',
       'competitor_names_Albertsons Cos.', 'competitor_names_Allentown',
       ...
       'industry_name_Transportation Equipment',
       'industry_name_Transportation and Logistics',
       'industry_name_Trucking, Truck Leasing',
       'industry_name_Utilities: Gas and Electric',
       'industry_name_Waste Management',
       'industry_name_Wholesalers: Diversified',
       'industry_name_Wholesalers: Electronics and Office Equipment',
       'industry_name_Wholesalers: Food and Grocery',
       'industry_name_Wholesalers: Health Care', 'Target'],
      dtype='object', length=1134)

In [100]:
# raw_df.head()
industries=raw_df['industry_name']
le=LabelEncoder()
# print(le.fit_transform(industries))
# print(industries)

target=le.fit_transform(industries)
cleansed_df['Target']=target
cleansed_df.head()

,Amount1,account_name,contacts,profile_id,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Akron,competitor_names_Albertsons Cos.,competitor_names_Allentown,...,industry_name_Transportation Equipment,industry_name_Transportation and Logistics,"industry_name_Trucking, Truck Leasing",industry_name_Utilities: Gas and Electric,industry_name_Waste Management,industry_name_Wholesalers: Diversified,industry_name_Wholesalers: Electronics and Office Equipment,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care,Target
0,50000,Walmart,['Eduardo Brayner'],0015I11484m7u87wBuD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
1,50000,Costco,['Karim Krzan'],0011S65285x2u62kQjM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,49
2,50000,Target,['Michael McNamara'],0018K30946c0u64tLmE,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,38
3,11861000,Cincinnati,['Lynette Lewey'],0012V84143a2u46dJuZ,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
4,50000,Menomonee Falls,['Karl Armstong'],0018O29856x4u54fHnE,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26


In [101]:
def standardise_amount(cleansed_df):
    df=cleansed_df.copy()
    df.Amount1=pd.to_numeric(df.Amount1)
    df.Amount1=((df.Amount1-df.Amount1.mean()).abs())/(df.Amount1.std())


    for col in df.columns:
        if df[col].dtype=='object':
            df.drop(col,axis=1,inplace=True)

    Y=df.Target
    X=df.drop('Target',axis=1)
    return X,Y

In [102]:
X,Y=standardise_amount(cleansed_df)

In [103]:
X.head()

,Amount1,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Akron,competitor_names_Albertsons Cos.,competitor_names_Allentown,competitor_names_Alphabet,competitor_names_Amazon.com,competitor_names_American Airlines Group,...,"industry_name_Toys, Sporting Goods",industry_name_Transportation Equipment,industry_name_Transportation and Logistics,"industry_name_Trucking, Truck Leasing",industry_name_Utilities: Gas and Electric,industry_name_Waste Management,industry_name_Wholesalers: Diversified,industry_name_Wholesalers: Electronics and Office Equipment,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care
0,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.393786,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
Y.head()

0    25
1    49
2    38
3    10
4    26
Name: Target, dtype: int64

# Profile similarity

### Without dynamically learnt distance metric 

In [134]:
similarity_scores=similarity_scorer(X,raw_df,'0015I11484m7u87wBuD',0.8,True)
# print(similarity_scores)

[('0015I11484m7u87wBuD', 0.9999999999999998), ('0017O90702b2u22mRuO', 0.9970387698474679), ('0010M31590u1u26mLrF', 0.996358227664311), ('0014A89474o7u08gAxF', 0.9951088655462809), ('0013C88074s8u10gLgP', 0.9943256153575845), ('0014O58104g1u89gMwI', 0.9939290185658288), ('0015A58652y5u85aTeR', 0.992605720993937), ('0014Q41571u4u83tPyT', 0.9919941049659444), ('0019S08787o6u53lIcW', 0.9919068248263776), ('0014X40021u0u07iDaG', 0.9917636565602805), ('0019I17871m1u56eHjB', 0.9913441572881068), ('0011G26362f4u65jMrJ', 0.9911302552549098), ('0012G45083i3u57xUnC', 0.9908647984885324), ('0010R66270s7u92aVcS', 0.9907191666487185), ('0014Q79691j4u04fIyL', 0.990716663325577), ('0010F18193q5u88qFcL', 0.9906548114978068), ('0010Z59426c3u52uAnH', 0.9894865151037372), ('0015E83544r6u40tJyJ', 0.9890171565458741), ('0018G04323o7u59fLiY', 0.9889988683062112), ('0015P76867d2u67bNnA', 0.9889354107775039), ('0013Y05490r7u46fXeF', 0.98854923344198), ('0017Q17137q8u22gPmX', 0.9873554354803002), ('0013Z22106r5

In [106]:
raw_df[raw_df['profile_id']=='0018I84600a7u13kRsG']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
34,50000,Janice Cerniglia,Cortex QuickStart(MachineLearning),Needs Identification,Dell Technologies,[],['Axel Whitsel'],"Computers, Office Equipment",0018I84600a7u13kRsG


In [107]:
raw_df[raw_df['profile_id']=='0018W57652c3u02cNyP']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
31,50000,Astrid Schmaus,Engage - QuickStart(Coversational AI),Prospecting,Citigroup,[],['Doug Fries'],Commercial Banks,0018W57652c3u02cNyP


In [140]:
raw_df[raw_df['profile_id']=='0015I11484m7u87wBuD']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,[],['Eduardo Brayner'],General Merchandisers,0015I11484m7u87wBuD


In [141]:
raw_df[raw_df['profile_id']=='0017O90702b2u22mRuO']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
2,21483000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Berkshire Hathaway,[],['Courtney Bodey'],Insurance: Property and Casualty (Stock),0017O90702b2u22mRuO


In [142]:
raw_df[raw_df['profile_id']=='0010M31590u1u26mLrF']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
973,50000,Silas Golik,Amplify - QuickStart(Process Insights),Confirming Value with Power,Denver,[],['Neil Burbidge'],"Building Materials, Glass",0010M31590u1u26mLrF


### With Large Margin Nearest Neighbour algorithm

In [133]:
# Takes around 15 mins for each run
similarity_scores=similarity_scorer_with_metric_learn(X,Y,raw_df,'0015I11484m7u87wBuD',0.3,'LMNN')
print(similarity_scores)

/Users/sswetank/anaconda3/lib/python3.6/site-packages/metric_learn/lmnn.py:62: UserWarning: use_pca does nothing for the python_LMNN implementation
  warnings.warn('use_pca does nothing for the python_LMNN implementation')


[('0015I11484m7u87wBuD', 1.0000000000000002), ('0014A89474o7u08gAxF', 0.47851361357724553), ('0017O90702b2u22mRuO', 0.46727433363126003), ('0018F19522b5u71kQdN', 0.37147146547001314), ('0011C69264p4u76xGqS', 0.3577782478598439), ('0012V01205p4u70wXrX', 0.35453136776685545), ('0016H55187a0u64bXlC', 0.3505128718355659), ('0012V33132n6u90bYsR', 0.33906018283997147), ('0010P92980y5u92pCdR', 0.33858465636427076), ('0010T21694h1u32kLvY', 0.33653434861599196), ('0010A03293e7u62oRnZ', 0.3365258828928387), ('0019C26341k1u73vGeF', 0.33484484838576895), ('0018G04323o7u59fLiY', 0.3332649646298048), ('0017X07975s1u85kPdM', 0.32756613823104086), ('0017B59136c4u05rGzB', 0.3233279389761064), ('0019Q98628h0u30eTxJ', 0.31708304577808827), ('0015E83544r6u40tJyJ', 0.3168668452438808), ('0016T39480x0u71kPlM', 0.31616785246480206), ('0015G59474o5u66rDyP', 0.31595093830450227), ('0015O83471n4u35oZwK', 0.31502735763569784), ('0014X88750f8u17sOyN', 0.31350557167858994), ('0010O36077k4u25iCzL', 0.30996903060094

In [40]:
raw_df[raw_df['profile_id']=='0014B28932p5u37rWgX']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
3,10089000,Paris Poma,Engage - Enterprise(Customer360),Prospecting,Apple,[],['Axel Whitsel'],"Computers, Office Equipment",0014B28932p5u37rWgX


In [41]:
raw_df[raw_df['profile_id']=='0017S92782b0u84nAaG']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
10,47280000,Kaydon Lambrakis,Engage - Enterprise(Conversational AI),Prospecting,Ford Motor,[],['Jeff Lemmer'],Motor Vehicles and Parts,0017S92782b0u84nAaG


In [42]:
raw_df[raw_df['profile_id']=='0017L02849e9u73xUpG']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
238,50000,Mauricio Spadlin,Cortex QuickStart(AI Develpoment),Negotiations,Kalamazoo,[],['Bill Kale'],Medical Products and Equipment,0017L02849e9u73xUpG


In [136]:
raw_df[raw_df['profile_id']=='0015I11484m7u87wBuD']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,[],['Eduardo Brayner'],General Merchandisers,0015I11484m7u87wBuD


In [137]:
raw_df[raw_df['profile_id']=='0014A89474o7u08gAxF']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
8,50000,Freddie Faustman,Cortex QuickStart(MachineLearning),Needs Identification,AT&T,[],['Ron Yttri'],Telecommunications,0014A89474o7u08gAxF


In [138]:
raw_df[raw_df['profile_id']=='0017O90702b2u22mRuO']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
2,21483000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Berkshire Hathaway,[],['Courtney Bodey'],Insurance: Property and Casualty (Stock),0017O90702b2u22mRuO


In [139]:
raw_df[raw_df['profile_id']=='0018F19522b5u71kQdN']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
73,19317000,Chloe Arrington,Amplify - Enterprise(Survellance Insights),Use Case Confirmed,Charter Communications,[],['Renee Weisel'],Telecommunications,0018F19522b5u71kQdN


### With Nearest Component Analysis algorithm

In [129]:
similarity_scores=similarity_scorer_with_metric_learn(X,Y,raw_df,'0015I11484m7u87wBuD',0.7,'NCA')
print(similarity_scores)

[('0015I11484m7u87wBuD', 1.0000000000000002), ('0011C69264p4u76xGqS', 0.9157087622278907), ('0010T21694h1u32kLvY', 0.8896918326431453)]


In [130]:
raw_df[raw_df['profile_id']=='0015I11484m7u87wBuD']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,[],['Eduardo Brayner'],General Merchandisers,0015I11484m7u87wBuD


In [131]:
raw_df[raw_df['profile_id']=='0011C69264p4u76xGqS']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
233,50000,Hosea Norsworthy,Cortex QuickStart(AI Develpoment),Negotiations,Plano,[],['June Linney'],General Merchandisers,0011C69264p4u76xGqS


In [132]:
raw_df[raw_df['profile_id']=='0010T21694h1u32kLvY']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
731,36033000,Elba Saylors,Amplify - Enterprise(Survellance Insights),Needs Identification,San Diego,[],['Celeste Ganino'],General Merchandisers,0010T21694h1u32kLvY


In [128]:
raw_df.head()

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,[],['Eduardo Brayner'],General Merchandisers,0015I11484m7u87wBuD
1,10580000,Astrid Schmaus,Amplify - Enterprise(Survellance Insights),Use Case Confirmed,Exxon Mobil,[],['Ridwan Zurawik'],Petroleum Refining,0017K23751y0u72fZpZ
2,21483000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Berkshire Hathaway,[],['Courtney Bodey'],Insurance: Property and Casualty (Stock),0017O90702b2u22mRuO
3,10089000,Paris Poma,Engage - Enterprise(Customer360),Prospecting,Apple,[],['Axel Whitsel'],"Computers, Office Equipment",0014B28932p5u37rWgX
4,9141000,Lisa Kalafatis,Cortex Enterprise(MachineLearning),Negotiations,UnitedHealth Group,[],['John Santelli'],Health Care: Insurance and Managed Care,0015R72262j6u33kWxP


### With LFDA

In [125]:
similarity_scores=similarity_scorer_with_metric_learn(X,Y,raw_df,'0014B28932p5u37rWgX',0.5,'LFDA')
print(similarity_scores)

,Amount1,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Akron,competitor_names_Albertsons Cos.,competitor_names_Allentown,competitor_names_Alphabet,competitor_names_Amazon.com,competitor_names_American Airlines Group,...,"industry_name_Toys, Sporting Goods",industry_name_Transportation Equipment,industry_name_Transportation and Logistics,"industry_name_Trucking, Truck Leasing",industry_name_Utilities: Gas and Electric,industry_name_Waste Management,industry_name_Wholesalers: Diversified,industry_name_Wholesalers: Electronics and Office Equipment,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care
0,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.393786,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0.568581,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2.363333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### With MLKR

In [24]:
similarity_scores=similarity_scorer_with_metric_learn(X,Y,raw_df,'0014B28932p5u37rWgX',0.5,'MLKR')
print(similarity_scores)

ValueError: shapes (999,1130) and (999,1130) not aligned: 1130 (dim 1) != 999 (dim 0)

In [82]:
X=np.random.random(4)

In [83]:
X

array([0.61461335, 0.21353609, 0.97281606, 0.01995753])

In [87]:
from sklearn.model_selection import LeavePOut,LeaveOneOut

In [88]:
lpo=LeavePOut(2)
loo=LeaveOneOut()

In [86]:
for train, test in lpo.split(X):
    print("%s %s"%(train,test))

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


In [90]:
for train,test in loo.split(X):
    print("%s %s"%(train,test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]
